参考：[一文教你如何用PyTorch构建 Faster RCNN](https://zhuanlan.zhihu.com/p/56710152)

In [ ]:
import torch
image = torch.zeros((1, 3, 800, 800)).float()

bbox = torch.FloatTensor([[20, 30, 400, 500], [300, 400, 500, 600]]) 

# [y1, x1, y2, x2] format
labels = torch.LongTensor([6, 8]) # 0 represents background
sub_sample = 16